In [2]:
from filtros import *
from functions import eta_subciclo
from functions import eta_subciclo_nominal
from fit_voc import *
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

### Obtenemos el Ground Truth de SOH de las celdas

In [3]:
# Obtenemos el ground truth
gt_path = "C:/Users/Bruno/OneDrive - Universidad de Chile/BGMG/CASE/Datasets/Dataset_aging/cells_SOH_25deg.csv"
gt_soh_values = pd.read_csv(gt_path)
gt_soh_values

,D30_0,E30_0,F30_0,C85_70,D85_70,D100_85,E100_85,F100_85,D100_0_drive,E100_0_drive,D100_0_,E100_0_
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.956672,0.945440,0.953341,0.978380,0.978223,0.985113,0.987026,0.984339,0.957386,0.952200,0.958063,0.957267
2,0.888100,0.880463,0.888987,0.969021,0.970280,0.965058,0.964104,0.964840,0.934058,0.935481,0.933245,0.934681
3,0.857320,0.851063,0.853256,0.960418,0.962952,0.946165,0.944715,0.946973,0.920934,0.922607,0.908312,0.910992
4,0.841922,0.835265,0.833843,0.953576,0.955763,0.927879,0.927477,0.930778,0.911324,0.912829,0.889487,0.895003
5,0.832690,0.825146,0.824182,0.947506,0.950879,0.913063,0.908689,0.917540,0.902280,0.903161,0.883462,0.886562
6,0.824490,0.814722,0.810584,0.941144,0.942437,0.884896,0.876411,0.896086,0.892396,0.893675,0.876218,0.878436
7,0.818153,0.801828,0.797390,0.937413,0.934023,0.887786,0.888055,0.898636,0.883509,0.884577,0.868952,0.871723
8,0.812961,0.796190,0.791492,0.930387,0.925096,0.857548,0.845812,0.885889,0.875958,0.876944,0.862589,0.865615
9,0.803136,0.781448,0.765043,0.928263,0.919983,0.854349,0.839055,0.881913,0.864434,0.866416,0.855277,0.858361


In [4]:
# Obtenemos los ciclos de cada celda
path = "C:/Users/Bruno/OneDrive - Universidad de Chile/BGMG/CASE/Datasets/Dataset_aging/cells_cycles_25deg.csv"
cycle_values = pd.read_csv(path)
cycle_values

,D30_0,E30_0,F30_0,C85_70,D85_70,D100_85,E100_85,F100_85,D100_0_drive,E100_0_drive,D100_0_,E100_0_
0,3361,3124,2616,6200,6199,5677,5677,5678,693,703,1170,1171


## Aplicamos el modelo de degradación a cada celda
### Definimos los modelos a utilizar

In [ ]:
param = dict(
    vL=1.35531394,
    v0=4.12017677,
    gamma=0.13286143,
    alpha=0.16945463,
    beta=2.34538224,
    Qmax=4.850,
    # Qmax=df.SoH.max(),
    Bias_Rint=0.01454392,
    Factor_Rint=0.1879029,
    life_cycles=300,
    adapt_cell=True
)

f_lineal = FiltrosAnidados()
f_lineal.Q_inst = param.get("Qmax")
f_lineal.fit_batt_model(param)
f_lineal.modelo_th.fit_inverse()
# Ajustamos los parámetros del estimaodor
f_lineal.voc_thresh = 0.05
f_lineal.voc_times = 47
f_lineal.sigma_autonomia = 0.04
f_lineal.sigma_capacidad = 0.001 * 5
f_lineal.estim_kwargs = {"sigma_e": 0.00075}

f_maxmin = FiltrosAnidados()
f_maxmin.Q_inst = param.get("Qmax")
f_maxmin.fit_batt_model(param)
f_maxmin.modelo_th.fit_inverse()
# Ajustamos los parámetros del estimaodor
f_maxmin.voc_thresh = 0.05
f_maxmin.voc_times = 47
f_maxmin.sigma_autonomia = 0.04
f_maxmin.sigma_capacidad = 0.001 * 5
f_maxmin.estim_kwargs = {"sigma_e": 0.00075}

f_subcnom = FiltrosAnidados()
f_subcnom.Q_inst = param.get("Qmax")
f_subcnom.fit_batt_model(param)
f_subcnom.modelo_th.fit_inverse()
# Ajustamos los parámetros del estimaodor
f_subcnom.voc_thresh = 0.05
f_subcnom.voc_times = 47
f_subcnom.sigma_autonomia = 0.04
f_subcnom.sigma_capacidad = 0.001 * 5
f_subcnom.estim_kwargs = {"sigma_e": 0.00075}

# f_subc = FiltrosAnidados()
# f_subc.Q_inst = param.get("Qmax")
# f_subc.fit_batt_model(param)
# f_subc.modelo_th.fit_inverse()
# # Ajustamos los parámetros del estimaodor
# f_subc.voc_thresh = 0.05
# f_subc.voc_times = 47
# f_subc.sigma_autonomia = 0.04
# f_subc.sigma_capacidad = 0.001 * 5
# f_subc.estim_kwargs = {"sigma_e": 0.00075}